In [1]:
import sys
sys.path.append('..')
import unittest
import os
import numpy as np
import pickle
from Compound_n import Compound
from ModificationSiteLocator import ModificationSiteLocator
from SiteLocator import SiteLocator
import utils

# library = "GNPS-LIBRARY"
library = "GNPS-MSMLS"
# library = "GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE"

# load data_dict_filtered
with open(os.path.join("../data/libraries",library,"data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

# load matches
with open(os.path.join("../data/libraries",library,"matches.pkl"), "rb") as f:
    matches = pickle.load(f)

# load cachedStructures_filtered
with open(os.path.join("../data/libraries",library,"cachedStructures.pkl"), "rb") as f:
    cachedStructures_filtered = pickle.load(f)



args = {
    'filter_peaks_method':"top_k",
      'filter_peaks_variable':60,
        'mz_tolerance':0.05,
        'mz_tolerance':0.05
}
peak_presence_only = False
consider_intensity = False
combine = False

In [11]:
m0 = "CCMSLIB00005464384"
m1 = "CCMSLIB00005464404"

In [12]:
main_compound = Compound(data_dict_filtered[m0], cachedStructures_filtered[m0], args)
modified_compound = Compound(data_dict_filtered[m1], cachedStructures_filtered[m1], args)
site_locator = ModificationSiteLocator(main_compound, modified_compound, args)
new_scores = site_locator.generate_probabilities(shifted_only=(not combine), PPO=peak_presence_only, CI=consider_intensity)
modification_site = utils.calculateModificationSites(modified_compound.structure, main_compound.structure, False)[0]

molMol = cachedStructures_filtered[m0]
modifMol = cachedStructures_filtered[m1]
molSmiles = data_dict_filtered[m0]['Smiles']
modifSmiles = data_dict_filtered[m1]['Smiles']
site = SiteLocator(data_dict_filtered[m0], data_dict_filtered[m1], molSmiles, args)

scores_unshifted, scores_shifted = site.calculate_score(peak_presence_only, consider_intensity)
old_scores = site.distance_score(scores_unshifted, scores_shifted, combine)
old_scores = old_scores - np.min(old_scores)
if np.sum(old_scores) != 0:
    old_scores = old_scores / np.sum(old_scores)
else:
    old_scores = np.zeros(len(old_scores))

here?
here?


In [4]:
print(len(main_compound.peaks))
print(len(site.molPeaks))

60
60


In [5]:
print(site.shifted)
print( site_locator.shifted)

[(24, 18), (50, 51), (46, 44), (36, 25), (16, 12), (42, 33), (28, 21), (44, 41), (23, 17), (56, 55), (26, 19), (51, 50), (32, 22), (43, 34), (35, 23), (25, 20), (37, 28)]
[(24, 18), (50, 51), (46, 44), (36, 25), (16, 12), (42, 33), (28, 21), (44, 41), (23, 17), (56, 55), (26, 19), (51, 50), (32, 22), (43, 34), (35, 23), (25, 20), (37, 28)]


In [6]:
scores_unshifted

array([ 9., 14., 14., 14.,  9.,  7.,  7., 12.])

In [7]:
scores_shifted

array([22., 54., 50., 59., 46., 37., 36., 44.])

In [8]:
s_peakids = [_[0] for _ in site_locator.shifted]
positive_contributions = site_locator.calculate_contributions(s_peakids, peak_presence_only, consider_intensity)
positive_contributions

[22, 54, 50, 59, 46, 37, 36, 44]

In [10]:
np.allclose(new_scores, old_scores)

True

In [123]:
# append 11 rectangles to eachother in svg

def draw_gradient_svg(length, diam, ax = 0, steps = 100):
    '''
    ax = 0: x axis
    ax = 1: y axis
    '''

    width = length
    height = diam
    if ax == 1:
        width = diam
        height = length
    
    svg = """<svg width="{}" height="{}" xmlns="http://www.w3.org/2000/svg">""".format(width, height)
    rectWidth = width/steps
    rectHeight = height
    if ax == 1:
        rectWidth = width
        rectHeight = height/steps


    # else:
    #     # add text to svg rotated
    #     svg += """<text x="{}" y="{}" fill="black" transform="rotate(90, {}, {})">{}</text>""".format(0, height/2, 0, height/2, "low likelihood")
    #     svg += """<text x="{}" y="{}" fill="black" transform="rotate(90, {}, {})">{}</text>""".format(0, width-100, 0, width-100, "high likelihood")

    for i in range(steps, -1, -1):
        if ax == 0:
            svg += """<rect x="{}" y="{}" width="{}" height="{}" fill="rgb({}, 0, {})"/>""".format(i*rectWidth, 0, rectWidth, rectHeight, (i/steps)*255, (1-i/steps)*255)
        else:
            svg += """<rect x="{}" y="{}" width="{}" height="{}" fill="rgb({}, 0, {})"/>""".format(0, i*rectHeight, rectWidth, rectHeight, (i/steps)*255, (1-i/steps)*255)

    if ax == 0:
    #     # add text to svg
        fontSize = 10
        svg += """<text x="{}" y="{}" fill="white">{}</text>""".format(5, (height+fontSize)/2, "low likelihood")
        # get width of text
        text = "high likelihood"
        svg += """<text x="{}" y="{}" fill="white">{}</text>""".format(width-len(text)*fontSize/1.42, (height+fontSize)/2, text)
    else:
        # add text to svg rotated
        fontSize = 10
        text = "high likelihood"
        svg += """<text x="{}" y="{}" fill="white" transform="rotate(90, {}, {})">{}</text>""".format(5, -(width-fontSize)/2, 0, 0, "low likelihood")
        svg += """<text x="{}" y="{}" fill="white" transform="rotate(90, {}, {})">{}</text>""".format(height-len(text)*fontSize/1.42, -(width-fontSize)/2, 0, 0, text)

    svg += "</svg>"
    return svg

# show svg
from IPython.display import SVG, display
print(draw_gradient_svg(1000, 100, ax = 0, steps = 100))
display(SVG(draw_gradient_svg(1000, 30, ax = 1, steps = 30)))



<svg width="1000" height="100" xmlns="http://www.w3.org/2000/svg"><rect x="1000.0" y="0" width="10.0" height="100" fill="rgb(255.0, 0, 0.0)"/><rect x="990.0" y="0" width="10.0" height="100" fill="rgb(252.45, 0, 2.5500000000000025)"/><rect x="980.0" y="0" width="10.0" height="100" fill="rgb(249.9, 0, 5.100000000000005)"/><rect x="970.0" y="0" width="10.0" height="100" fill="rgb(247.35, 0, 7.650000000000007)"/><rect x="960.0" y="0" width="10.0" height="100" fill="rgb(244.79999999999998, 0, 10.20000000000001)"/><rect x="950.0" y="0" width="10.0" height="100" fill="rgb(242.25, 0, 12.75000000000001)"/><rect x="940.0" y="0" width="10.0" height="100" fill="rgb(239.7, 0, 15.300000000000013)"/><rect x="930.0" y="0" width="10.0" height="100" fill="rgb(237.15, 0, 17.849999999999987)"/><rect x="920.0" y="0" width="10.0" height="100" fill="rgb(234.60000000000002, 0, 20.39999999999999)"/><rect x="910.0" y="0" width="10.0" height="100" fill="rgb(232.05, 0, 22.949999999999992)"/><rect x="900.0" y="0" 